In [1]:
# Clone and compile trec_eval
!git clone https://github.com/usnistgov/trec_eval.git
!cd trec_eval && make


Cloning into 'trec_eval'...
remote: Enumerating objects: 1147, done.
remote: Counting objects: 100% (332/332), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 1147 (delta 264), reused 277 (delta 226), pack-reused 815 (from 1)
Receiving objects: 100% (1147/1147), 764.18 KiB | 2.32 MiB/s, done.
Resolving deltas: 100% (769/769), done.
gcc -g -I.  -Wall -Wno-macro-redefined -DVERSIONID=\"10.0-rc2\"  -o trec_eval trec_eval.c formats.c meas_init.c meas_acc.c meas_avg.c meas_print_single.c meas_print_final.c gain_init.c get_qrels.c get_trec_results.c get_prefs.c get_qrels_prefs.c get_qrels_jg.c form_res_rels.c form_res_rels_jg.c form_prefs_counts.c utility_pool.c get_zscores.c convert_zscores.c measures.c  m_map.c m_P.c m_num_q.c m_num_ret.c m_num_rel.c m_num_rel_ret.c m_gm_map.c m_Rprec.c m_recip_rank.c m_bpref.c m_iprec_at_recall.c m_recall.c m_Rprec_mult.c m_utility.c m_11pt_avg.c m_ndcg.c m_ndcg_cut.c m_Rndcg.c m_ndcg_rel.c m_binG.c m_G.c m_rel_P.c m_success.c m_inf

In [7]:
# Import pandas and load your result files
import pandas as pd

# Load the result files, skipping header row
results_title_text_df = pd.read_csv('/content/Results_title_text.txt', sep='\s+', header=0, names=["query_id", "Q0", "doc_id", "rank", "score", "tag"])

# Clean the query_id columns
for df in [results_title_text_df]:
    df['query_id'] = df['query_id'].astype(str).str.replace('Q0-', '', regex=False)

# Save cleaned versions without headers
results_title_text_df.to_csv('/content/cleaned_Results_title_text.txt', sep=' ', index=False, header=False)


# ⬇Evaluate each cleaned result file using trec_eval
for label, cleaned_file in {
    "Title + Text": "cleaned_Results_title_text.txt",
    "MiniLM Rerank": "cleaned_Results_miniLM.txt",
    "USELM Rerank": "Results_useLM.txt",
    "doc2vec Rerank": "Results_doc2vec.txt"
}.items():
    print(f"\n{label}")
    !cd trec_eval && ./trec_eval /content/test.qrel /content/{cleaned_file}




Title + Text
runid                 	all	a06e8ec5-c2ec-443b-8cc5-142febf16054
num_q                 	all	300
num_ret               	all	29933
num_rel               	all	339
num_rel_ret           	all	304
map                   	all	0.4842
gm_map                	all	0.1139
Rprec                 	all	0.3679
bpref                 	all	0.8949
recip_rank            	all	0.4957
iprec_at_recall_0.00  	all	0.4959
iprec_at_recall_0.10  	all	0.4959
iprec_at_recall_0.20  	all	0.4959
iprec_at_recall_0.30  	all	0.4958
iprec_at_recall_0.40  	all	0.4936
iprec_at_recall_0.50  	all	0.4915
iprec_at_recall_0.60  	all	0.4915
iprec_at_recall_0.70  	all	0.4894
iprec_at_recall_0.80  	all	0.4771
iprec_at_recall_0.90  	all	0.4733
iprec_at_recall_1.00  	all	0.4733
P_5                   	all	0.1347
P_10                  	all	0.0773
P_15                  	all	0.0556
P_20                  	all	0.0430
P_30                  	all	0.0306
P_100                 	all	0.0101
P_200                 	all	0.0051
P_500         